# Dependent or independent ?

## Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.distance import vincenty
from sklearn.ensemble import ExtraTreesRegressor
import warnings
warnings.filterwarnings('ignore')

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_train = pd.read_csv('/Users/maelfabien/Desktop/LocalDB/RSSI/df_mess_train_3.csv')
df_test = pd.read_csv('/Users/maelfabien/Desktop/LocalDB/RSSI/df_mess_test_3.csv')

In [3]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)

## Error Functions

In [4]:
def vincenty_vec(vec_coord):
    vin_vec_dist = np.zeros(vec_coord.shape[0])
    if vec_coord.shape[1] !=  4:
        print('ERROR: Bad number of columns (shall be = 4)')
    else:
        vin_vec_dist = [vincenty(vec_coord[m,0:2],vec_coord[m,2:]).meters for m in range(vec_coord.shape[0])]
    return vin_vec_dist

In [5]:
# evaluate distance error for each predicted point
def Eval_geoloc(y_train_lat , y_train_lng, y_pred_lat, y_pred_lng):
    y_pred_lat[np.where(y_pred_lat>90)[0]]=90
    y_pred_lat[np.where(y_pred_lat<-90)[0]]=-90
    vec_coord = np.array([np.array(y_train_lat) , np.array(y_train_lng), y_pred_lat, y_pred_lng])
    err_vec = vincenty_vec(np.transpose(vec_coord))
    
    return err_vec

## k-Fold Cross Validation

In [6]:
df_train['did'].unique()

array([ 473335.,  473953.,  476512.,  476286.,  473438.,  476185.,
        476285.,  476314.,  476306.,  476317.,  476320.,  473502.,
        476318.,  476197.,  476312.,  476316.,  476251.,  476308.,
        473683.,  473796.,  476323.,  476329.,  476321.,  476324.,
        476332.,  476322.,  473864.,  473512.,  473805.,  476327.,
        476325.,  476515.,  476505.,  476517.,  476212.,  476503.,
        476507.,  476611.,  476610.,  476606.,  476604.,  476607.,
        476602.,  476609.,  476600.,  476598.,  476615.,  476521.,
        476525.,  476523.,  476830.,  474181.,  476828.,  474126.,
        476826.,  476853.,  476315.,  474176.,  476852.,  476987.,
        476307.,  476868.,  476833.,  476861.,  473897.,  476276.,
        473902.,  476275.,  476888.,  476891.,  476274.,  476280.,
        476257.,  476210.,  476231.,  474192.,  476225.,  476256.,
        476161.,  476228.,  476835.,  473368.,  473892.,  476883.,
        476889.,  476884.,  476887.,  473899.,  476885.,  4762

For independent risks :

In [7]:
error_independent = []

for device in df_train['did'].unique() :
    
    df_train_lov = df_train[df_train.did != device]
    df_test_lov = df_train[df_train.did == device]
    
    X_train = df_train_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_train = df_train_lov['lat']
    y_lng_train = df_train_lov['lng']
    
    X_test = df_test_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_test = df_test_lov['lat']
    y_lng_test = df_test_lov['lng']
    
    clf_lng = ExtraTreesRegressor(n_estimators=10)
    clf_lng.fit(X_train, y_lng_train)
    pred_lng = clf_lng.predict(X_test)
    
    clf_lat = ExtraTreesRegressor(n_estimators=10)
    clf_lat.fit(X_train, y_lat_train)
    pred_lat = clf_lat.predict(X_test)
    
    err_vec = Eval_geoloc(y_lat_test , y_lng_test, pred_lat, pred_lng)
    err = np.percentile(err_vec, 80)
    error_independent.append(err)
    
    print(err)

8506.389972690758
2338.205181275432
5016.020177466543
6282.637504753409
4381.391747285404
2730.0934563396895
5794.274217200038
2889.266123535241
4149.355655488064
3933.0659422543067
1213.4726372352388
1434.3518985325247
2125.1395131694335
1414.343711302411
5438.336698716719
7408.798509752402
7925.505208504677
2554.69570424476
2675.121069906966
4745.074260797381
6161.346712644995
263.81471004227456
1655.1339096718405
1452.0760015369556
1645.8538686069362
1612.767849280381
5960.053580147819
2772.546277384142
4866.67906032537
13333.408484080808
8288.034475271315
3065.665157658542
1895.5177087197383
2085.0250225380532
3495.274030165019
8707.925720519255
332.46500478946257
1866.950762784308
13696.708455416836
5199.6024634149835
5598.601993620925
2737.981800926037
1344.6777235306656
2349.9928082257748
4633.2214484258375
97867.81423814905
2971.417205193782
1863.1553377807165
6308.723969376792
475.59003266132436
8127.340151880876
7740.726433880112
5011.1005008939965
2090.41344422206
1846.17736

For dependent risks, by predicting first the longitude :

In [8]:
error_lng = []

for device in df_train['did'].unique() :
    
    df_train_lov = df_train[df_train.did != device]
    df_test_lov = df_train[df_train.did == device]
    
    X_train = df_train_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_train = df_train_lov['lat']
    y_lng_train = df_train_lov['lng']
    
    X_test = df_test_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_test = df_test_lov['lat']
    y_lng_test = df_test_lov['lng']
    
    clf_lng = ExtraTreesRegressor(n_estimators=10)
    clf_lng.fit(X_train, y_lng_train)
    pred_lng = clf_lng.predict(X_test)
    
    clf_lat = ExtraTreesRegressor(n_estimators=10)
    new_X_train = pd.concat([X_train, y_lng_train], axis=1)
    
    clf_lat.fit(new_X_train, y_lat_train)
    new_X = pd.concat([X_test.reset_index(drop=True), pd.DataFrame(pred_lng).reset_index(drop=True)], axis=1)
    pred_lat = clf_lat.predict(new_X)
    
    err_vec = Eval_geoloc(y_lat_test , y_lng_test, pred_lat, pred_lng)
    err = np.percentile(err_vec, 80)
    error_lng.append(err)
    
    print(err)

8514.149729082863
2480.035206815076
5034.815931213441
6214.926941073938
3626.0626685437555
2648.2490853325544
5414.971924781733
3224.137245163359
690.5500877530941
2276.3461725905468
1444.4353610569447
972.4088055519364
2039.5580813068789
210.8566080924113
5029.2611538201445
7976.809205573988
8152.856502228019
326.0755326384602
2137.559056046669
4655.7219691947885
6598.087088311751
567.8440570884813
2537.086666480679
2682.6944873602038
2115.047475238517
876.3493148051223
5398.649654664541
3150.9769743056054
4609.687062223649
13075.163921796368
8160.807868095084
2492.408820371652
2707.924485932207
2868.0367924159436
3024.5289157212474
7890.8033069224
336.53204509831744
561.5131494367018
2381.4887173158177
4722.021616636812
6783.806568946557
2033.240081274378
1697.4127346161213
2434.4115297166354
3505.1016668124207
105376.87740649584
3658.6847250205824
1765.9812869934558
4871.146082869977
335.74894114305505
7905.592425042541
8857.89620529527
5716.237281401454
2314.1722471224334
1596.2663

For dependent risks, by predicting first the latitude :

In [9]:
error_lat = []

for device in df_train['did'].unique() :
    
    df_train_lov = df_train[df_train.did != device]
    df_test_lov = df_train[df_train.did == device]
    
    X_train = df_train_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_train = df_train_lov['lat']
    y_lng_train = df_train_lov['lng']
    
    X_test = df_test_lov.drop(['messid', 'lat', 'lng', 'did'], axis=1)
    y_lat_test = df_test_lov['lat']
    y_lng_test = df_test_lov['lng']
    
    clf_lat = ExtraTreesRegressor(n_estimators=10)
    clf_lat.fit(X_train, y_lat_train)
    pred_lat = clf_lat.predict(X_test)
    
    clf_lng = ExtraTreesRegressor(n_estimators=10)
    new_X_train = pd.concat([X_train, y_lat_train], axis=1)
    
    clf_lng.fit(new_X_train, y_lng_train)
    new_X = pd.concat([X_test.reset_index(drop=True), pd.DataFrame(pred_lat).reset_index(drop=True)], axis=1)
    
    pred_lng = clf_lng.predict(new_X)
    
    err_vec = Eval_geoloc(y_lat_test , y_lng_test, pred_lat, pred_lng)
    err = np.percentile(err_vec, 80)
    error_lat.append(err)
    
    print(err)

8062.942681543891
2365.171346841827
5489.596436606539
3682.469660852621
4371.719032317516
2911.401867160183
5573.397993357904
2418.054375591671
2865.3972387927124
4345.478506548134
1683.8323162845045
2418.2630267449995
1970.5751365250296
1087.2673612565548
5089.509542293505
7694.7110199294275
7928.533633769798
2560.2487606646537
2184.4153104136585
5055.07546077454
4893.689609438945
592.3627055027763
1303.9771315959424
1874.0922710940506
1308.0242400982115
485.9087771246809
3520.6583607438597
3307.4883704645626
5216.946697508269
12436.008371175436
7789.434523341875
3533.286657427201
2755.0846529856813
2703.394241505292
3072.902290667191
9210.236223837515
260.036711667649
1035.062874029424
4500.098427937415
4324.292037220446
4213.548836078704
2875.064884389412
2316.2579429662746
2616.210753058131
4427.158002864031
90331.08828261866
992.1914920589159
1762.9137670237565
6723.697203226802
400.0888882862955
7974.55435989609
8153.839860236269
5053.429058530915
2326.4384159445535
2129.02936634

In [12]:
print("Independant : " + str(np.median(np.array(error_independent))))
print("Longitude First : " + str(np.median(np.array(error_lng))))
print("Latitude First : " + str(np.median(np.array(error_lat))))

Independant : 2971.417205193782
Longitude First : 2991.333516394935
Latitude First : 3072.902290667191


The best model appears to be : Estimating longitude first, and then latitude in the dependent model.